In [17]:
import os
import glob
import numpy as np
import pandas as pd
#!pip install SimpleITK
import SimpleITK as sitk
from dltk.io import preprocessing
from skimage import filters
import nipype.interfaces.fsl as fsl

import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt

import os

## 1. Preprocessing

In [ ]:
# 1. Resample image

def resample(image, output_spacing = [2.0, 2.0, 2.0]):
    ''' Resample images to 2-mm isotropic voxels
      
        Parameters:
            image -- Image in SimpleITK format
            output_spacing -- Target space representation of each voxel
            
        Returns:
            Output image in SimpleITK format
    '''
    
    # Resample images to 2mm spacing
    original_spacing = image.GetSpacing()
    original_size = image.GetSize()

    output_size = [
        int(np.round(original_size[0] * (original_spacing[0] / output_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / output_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / output_spacing[2])))]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(output_spacing)
    resample.SetSize(output_size)
    resample.SetOutputDirection(image.GetDirection())
    resample.SetOutputOrigin(image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(image.GetPixelIDValue())

    resample.SetInterpolator(sitk.sitkBSpline)

    return resample.Execute(image)


# 2. Registrate image

def register(sitk_fixed, sitk_moving, bspline = False):
    ''' register image with SimpleElastix using affine transformation.
        
        Parameters:
            sitk_fixed -- Reference atlas
            sitk_moving -- Image to be registrated 
            bspline -- Whether or not to perform non-rigid registration
    '''
    
    elastixImageFilter = sitk.ElastixImageFilter()
    elastixImageFilter.SetFixedImage(sitk_fixed)
    elastixImageFilter.SetMovingImage(sitk_moving)

    parameterMapVector = sitk.VectorOfParameterMap()
    parameterMapVector.append(sitk.GetDefaultParameterMap("affine"))
    
    if bspline:
        parameterMapVector.append(sitk.GetDefaultParameterMap("bspline"))
        
    elastixImageFilter.SetParameterMap(parameterMapVector)

    elastixImageFilter.Execute()
    
    return elastixImageFilter.GetResultImage()


# Function for registering image and saving them to a new directory

def register_image(file_path, atlas):
    ''' Parameters:
            file_path -- where file is located, with the filename
            atlas -- Reference sitk image for registration
    '''
        
    filename = file_path.split('/')[-1] # filename without path
    
    # extract the image ID
    image_ID = filename.strip().split('_')[-1][1:-4]
        
    # change the ID into a int64 numpy variable for indexing
    image_ID = np.int64(image_ID)
        
    # extract image label using image_ID
    row_index = description.index[description['Image Data ID'].str[1:].astype('int') == image_ID].tolist()[0]
    # obtain the corresponding row in the dataframe
    row = description.iloc[row_index]
    # get the label
    label = row['Group']
    
    # load sitk image
    sitk_moving = sitk.ReadImage(file_path)
    sitk_moving = resample(sitk_moving)
    registrated = register(atlas, sitk_moving)
    
    # prepare the destination path
    new_dir = os.path.join(REGISTRATED, 
                           label,
                           filename)
    sitk.WriteImage(registrated, new_dir)


In [ ]:
# Raw data path
filenames = glob.glob('/Users/seoyeonhong/Desktop/BIOST527/ADNI/RAW/*/*/*/*/*.nii')

# metadata path
description = pd.read_csv('/Users/seoyeonhong/Desktop/BIOST527/ADNI/ADNI_description.csv')

# Atlas (reference for registering image)
IMAGE = '/Users/seoyeonhong/Desktop/BIOST527/ADNI/average305_t1_tal_lin.nii'
atlas = sitk.ReadImage(IMAGE)
atlas = resample(atlas)

# New destination
REGISTERED = '/Users/seoyeonhong/Desktop/BIOST527/ADNI/REGISTERED/'
# Images will be divided by label
REGISTERED_SUBFOLDERS = ['AD/', 'MCI/', 'CN/']

In [ ]:
# this is for registering images
#### unfortunately there were problems in dependencies and installed python 3.5 to use ElastixImageFilter 
#### then wiped python so cannot run it again not installed in current python 3.8
# for file in filenames:
#     register_image(file, atlas)

In [ ]:
def skull_strip_nii(original_img, destination_img, frac = 0.2):
    ''' Uses FSL-BET 
        (https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/BET/UserGuide#Main_bet2_options:)
        
        Parameters:
            original_img -- Original nii image
            destination_img -- The new skull-stripped image
            frac -- Fractional intensity threshold for BET
    '''
    
    btr = fsl.BET()
    btr.inputs.in_file = original_img
    btr.inputs.frac = frac
    btr.inputs.out_file = destination_img
    btr.cmdline
    res = btr.run()


In [ ]:
# database locations
SKULL_STRIPPED_DB = '/Users/seoyeonhong/Desktop/BIOST527/ADNI/SKULL_STRIPPED/'
SUB_FOLDERS = ['AD/', 'MCI/', 'CN/']

exceptions = []

for folder in SUB_FOLDERS:
    registered_filenames = glob.glob(f"{REGISTERED}{folder}/*.nii")
    output_dir = SKULL_STRIPPED_DB + folder
    for filename in registered_filenames: 
        try:
            skull_strip_nii(filename, output_dir + filename.split('/')[-1], frac = 0.2)
        except RuntimeError:
            exceptions.append(filename)

In [2]:
# shape of the images
IMG_SHAPE = (78, 110, 86)
# IMG_2D_SHAPE = (IMG_SHAPE[1] * 4, IMG_SHAPE[2] * 4)

# def slices_matrix_2D(img):
#     ''' Transform a 3D MRI image into a 2D image by obtaining 16 horizontal slices
#         and assembling on a 4x4 two-dimensional grid. Returns 2D image
        
#         Parameters:
#             img -- np.ndarray with the 3D image
#     '''

#     # create final 2D image 
#     image_2D = np.empty(IMG_2D_SHAPE)

#     # set limits and step
#     TOP = 60
#     BOTTOM = 30
#     STEP = 2
#     N_CUTS = 16

#     # iterator for the cuts
#     cut_it = TOP
#     # iterator for the rows of the 2D final image
#     row_it = 0
#     # iterator for the columns of the 2D final image
#     col_it = 0

#     # cut
#     for cutting_time in range(N_CUTS):
#         cut = img[cut_it, :, :]
#         cut_it -= STEP

#     # reset the row iterator and move the
#     # col iterator when needed
#     if cutting_time in [4, 8, 12]:
#         row_it = 0
#         col_it += cut.shape[1]

#     # copy the cut to the 2D image
#     for i in range(cut.shape[0]):
#         for j in range(cut.shape[1]):
#             image_2D[i + row_it, j + col_it] = cut[i, j]
#     row_it += cut.shape[0]
  
#     # return the final 2D image, with 3 channels
#     return np.repeat(image_2D[None, ...], 3, axis=0).T

def load_image(abs_path, whitening = True, convert_to_2D = False):
    ''' Returns 3D image transformed into 2D along with its label given absolute path'''

    # obtain the label from the path (it is the last directory name)
    label = abs_path.split('/')[-2]

    # load the image with SimpleITK
    sitk_image = sitk.ReadImage(abs_path)

    # transform into a numpy array
    img = sitk.GetArrayFromImage(sitk_image)
    
    # apply whitening
    if whitening == True:
        img = preprocessing.whitening(img)

    # make the 2D image
    if convert_to_2D == True:
        img = slices_matrix_2D(img)
#     print(img.shape)

    return img, label

In [ ]:
# Transform skull stripped images into 2D and organize final dataset

SKULL_STRIPPED_DB = '/Users/seoyeonhong/Desktop/BIOST527/ADNI/SKULL_STRIPPED/'
SUB_FOLDERS = ['AD', 'MCI', 'CN']

imgs = []
labels = []
ids = []
filenames = []

for folder in SUB_FOLDERS:
    skullless_filenames = glob.glob(f"{SKULL_STRIPPED_DB}{folder}/*.nii")
    filenames.extend(skullless_filenames)
    for filename in skullless_filenames: 
        img, label = load_image(filename)
        imgs.append(img)
        labels.append(label)
        ids.append(filename.split('/')[-1].strip().split('_')[-1][1:-4])

In [ ]:
# Assemble final dataset
final_df = pd.DataFrame(zip(ids, imgs, labels, filenames), columns = ['ID', 'IMG', 'LABEL', 'DIR'])

In [ ]:
final_df.to_csv('preprocessed_images.csv')

## 2. Transforming to TFRecords Format

In [18]:
# Implement methods to build a tf.train.Feature from a basic python or numpy datatype.

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))
  
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [21]:
# Folder path for registrated and skull-stripped images
DB_REG_PATH = '/Users/seoyeonhong/Desktop/BIOST527/ADNI/REGISTERED/'
DB_SS_PATH = '/Users/seoyeonhong/Desktop/BIOST527/ADNI/SKULL-STRIPPED/'

# the data description file
DESCRIPTION_FILE = '/Users/seoyeonhong/Desktop/BIOST527/ADNI/ADNI_description.csv'

# data subfolders
CLASS_SUBFOLDERS = ['MCI/', 'AD/', 'CN/']

# supervised TFRecords database
DB_TF_PATH = '/Users/seoyeonhong/Desktop/BIOST527/ADNI/TFRecords3D/'
# tfrecords files - registered then skull stripped
TFREC_SS_TRAIN = 'train.3D.skull_stripped.tfrecords'
TFREC_SS_TEST = 'test.3D.skull_stripped.tfrecords'
TFREC_SS_VAL = 'validation.3D.skull_stripped.tfrecords'

In [22]:
# label mapping
LABELS = {'CN': 0, 'MCI': 1, 'AD': 2}

# shape of the images
IMG_SHAPE = (78, 110, 86)

In [23]:
# Defining the size of test and validation sets
# When using TFRecords data needs to be split into separate files initially

TEST_SPLIT = 0.15
VALIDATION_SPLIT = 0.15

In [24]:
# Shuffle and split train test data

In [25]:
# Fetch filenames as array
filenames = np.array(glob.glob('/Users/seoyeonhong/Desktop/BIOST527/ADNI/SKULL_STRIPPED/*/*.nii'))
print(len(filenames))

511


In [26]:
# shuffle and split data

for i in range(1000):
    np.random.shuffle(filenames)
    
test_margin = int(len(filenames) * TEST_SPLIT)
training_set, test_set = filenames[test_margin:], filenames[:test_margin]

validation_margin = int(len(training_set) * VALIDATION_SPLIT)
training_set, validation_set = training_set[validation_margin:], training_set[:validation_margin]

print('Training set:', training_set.shape)
print('Validation set:', validation_set.shape)
print('Test set:', test_set.shape)

Training set: (370,)
Validation set: (65,)
Test set: (76,)


In [27]:
train_tfrec = DB_TF_PATH + TFREC_SS_TRAIN
test_tfrec = DB_TF_PATH + TFREC_SS_TEST
val_tfrec = DB_TF_PATH + TFREC_SS_VAL

In [31]:
from PIL import Image

def create_tf_record(img_filenames, tf_rec_filename, labels):
    """Create a TFRecord file after converting image to 2D"""
    
    # open the file
    writer = tf.io.TFRecordWriter(tf_rec_filename)
    
    # iterate through all .nii files
    for file in img_filenames:
        # load the image and label
        img, label = load_image(file)
#         print(label)
#         # Verifying image
#         im = Image.fromarray(img.ravel())
#         if im.mode != 'RGB':
#             im = im.convert('RGB')
#         im.save(f"filename_{file.split('/')[-1].split('.')[0]}.png")
        
        # create features
        features = {'label': _int64_feature(labels[label]),
                    'image': _float_feature(img.ravel())}
        
        # create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature = features))

        # serialize to string and write on the file
        writer.write(example.SerializeToString())
        
    writer.close()

In [33]:
create_tf_record(training_set, train_tfrec, LABELS)
create_tf_record(validation_set, val_tfrec, LABELS)
create_tf_record(test_set, test_tfrec, LABELS)

In [16]:
n_training_samples = 0
for record in tf.compat.v1.io.tf_record_iterator(train_tfrec):
    n_training_samples += 1
print ('Training samples:', n_training_samples)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Training samples: 370
